In [1]:
import math
import numpy as np
import h5py
# import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
import json
import random
from collections import OrderedDict

# %matplotlib inline
# np.random.seed(0)

/root/miniconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# import cv2

In [3]:
EPOCHS = 10
BATCH_SIZE = 32
IMG_SIZE = 224
data_file = "dataset.txt"

In [4]:
weight_file ="vgg16_weights.npz"
weights = np.load(weight_file)

In [5]:
keys = sorted(weights.keys())
for index, key in enumerate(keys):
    print(index, key, np.shape(weights[key]))

0 conv1_1_W (3, 3, 3, 64)
1 conv1_1_b (64,)
2 conv1_2_W (3, 3, 64, 64)
3 conv1_2_b (64,)
4 conv2_1_W (3, 3, 64, 128)
5 conv2_1_b (128,)
6 conv2_2_W (3, 3, 128, 128)
7 conv2_2_b (128,)
8 conv3_1_W (3, 3, 128, 256)
9 conv3_1_b (256,)
10 conv3_2_W (3, 3, 256, 256)
11 conv3_2_b (256,)
12 conv3_3_W (3, 3, 256, 256)
13 conv3_3_b (256,)
14 conv4_1_W (3, 3, 256, 512)
15 conv4_1_b (512,)
16 conv4_2_W (3, 3, 512, 512)
17 conv4_2_b (512,)
18 conv4_3_W (3, 3, 512, 512)
19 conv4_3_b (512,)
20 conv5_1_W (3, 3, 512, 512)
21 conv5_1_b (512,)
22 conv5_2_W (3, 3, 512, 512)
23 conv5_2_b (512,)
24 conv5_3_W (3, 3, 512, 512)
25 conv5_3_b (512,)
26 fc6_W (25088, 4096)
27 fc6_b (4096,)
28 fc7_W (4096, 4096)
29 fc7_b (4096,)
30 fc8_W (4096, 1000)
31 fc8_b (1000,)


In [6]:
def conv_layer(input_data,filter_size,stride_size,pad="SAME",name="conv",weight_name="W1",bias_name="b1"):
    print("creating layer :"+name)
    with tf.name_scope(name):
        W = tf.Variable(weights[name+"_W"],name="weight")
        b = tf.Variable(weights[name+"_b"],name="bias")
        
        conv = tf.nn.conv2d(input_data, W, strides=stride_size, padding=pad)
        out = tf.nn.bias_add(conv, b)
        act = tf.nn.relu(out)
        tf.summary.histogram("weights", W)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

In [7]:
def vggnet_v1(input_data):
    print("creating model")
    output=[]
    layer_input = input_data
    
    print("reading vgg16 json file")
    vgg = json.load(open("vgg16.json"))
   
    for index,layer in enumerate(vgg):
        if "conv" in layer:
            output = conv_layer(layer_input,vgg[layer]["weights"],vgg[layer]["stride"],vgg[layer]["pad"],layer,"weight"+str(index),"bias"+str(index))
        elif "pool" in layer:
            output = tf.nn.max_pool(layer_input, ksize=vgg[layer]["ksize"], strides=vgg[layer]["strides"], padding=vgg[layer]["padding"],name=layer)
        layer_input = output
    
    pool5_flat = tf.contrib.layers.flatten(output)

    print("creating layer : fc6")
    with tf.name_scope('fc6'):
        fc6_W = tf.Variable(weights["fc6_W"],name="fc6_W")
        fc6_b= tf.Variable(weights["fc6_b"],name="fc6_b")

        fc6 = tf.nn.bias_add(tf.matmul(pool5_flat, fc6_W), fc6_b)
        fc6 = tf.nn.relu(fc6)
    
#     fc6 = tf.contrib.layers.fully_connected(P3,4096, weights_initializer=fc6_W, biases_initializer=fc6_b, scope="fc6")
#     tf.summary.histogram("fc6/relu", fc6)
    
    dropout1 = tf.layers.dropout(inputs=fc6, rate=0.5)

    print("creating layer : fc7")
    with tf.name_scope('fc7'):
        fc7_W = tf.Variable(weights["fc7_W"],name="fc7_W")
        fc7_b= tf.Variable(weights["fc7_b"],name="fc7_b")

        fc7 = tf.nn.bias_add(tf.matmul(dropout1, fc7_W), fc7_b)
        fc7 = tf.nn.relu(fc7)
    
#     fc7 = tf.contrib.layers.fully_connected(dropout1,4096, weights_initializer=fc7_W, biases_initializer=fc7_b, scope="fc7")
#     tf.summary.histogram("fc7/relu", fc7)
    
    dropout2 = tf.layers.dropout(inputs=fc7, rate=0.5)
    
    print("creating layer : fc8")
    fc8 = tf.contrib.layers.fully_connected(dropout2,2, biases_initializer=tf.constant_initializer(1.0), activation_fn=None, scope="fc8")
    tf.summary.histogram("fc8", fc8)
    
    
    print("model completed.")
    ##NEED FULLY CONNECTED LAYER
    return fc8

In [8]:
def prepare_data(data):
    newData=[]
    labels=[]
    print("preparing data....")
    for sample in data:
        img_path,label = sample.strip().split(" ")
        img = Image.open(img_path)
        img = img.resize((IMG_SIZE,IMG_SIZE))
        img = np.array(img)
        img = img/255
        newData.append(img)
        if "cat" in img_path:
            labels.append(np.array([1,0]))
        elif "dog" in img_path:
            labels.append(np.array([0,1]))
    print("preparing data completed.")
    return np.array(newData),np.array(labels)

In [9]:
f = open(data_file,"r")
data = f.readlines()
random.shuffle(data)
print(len(data))
train_data = data[:20000]
val_data = data[20000:]

25000


In [ ]:
tf.reset_default_graph()
learning_rate = 0.001

with tf.name_scope('inputs'):
    X = tf.placeholder(tf.float32, shape=(None,IMG_SIZE,IMG_SIZE , 3),name="X")
    tf.summary.image('input', X, BATCH_SIZE)
    Y = tf.placeholder(tf.float32,shape=(None,2),name="labels")
    
logits = vggnet_v1(X)
with tf.name_scope("loss"):    
    train_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y), name="loss")
    tf.summary.scalar("loss", train_loss)

with tf.name_scope("train"):
    #optimizer = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(train_loss)
    optimizer = tf.train.AdamOptimizer(learning_rate).minimize(train_loss)
print("optimizer set")

with tf.name_scope("accuracy"):
    correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    tf.summary.scalar("accuracy", accuracy)
print("accuracy set")

merged_summary = tf.summary.merge_all()

# init = 
# print("initializing the variables")

with tf.Session() as sess:
    print("session created.")
    print("initializing global variables")
    sess.run(tf.global_variables_initializer())
    
    print("writing tensorboard")
    writer = tf.summary.FileWriter("/tmp/tensorboard/cat-dog-vgg0")
    writer.add_graph(sess.graph)
    print("writer added to graph")
    
#     for epoch in range(EPOCHS):
        
#         print("processing epoch "+str(epoch))
#         print("total number of batches in 1 epoch: "+ str(len(train_data)/BATCH_SIZE))
        
#         for i in range(0,len(train_data),BATCH_SIZE): 
# #             print(i)
#             print("Batch:" + str((i/BATCH_SIZE)+1))
            
#             minibatch_X, minibatch_Y = prepare_data(train_data[i:i+BATCH_SIZE])
#             #sess.run(optimizer,feed_dict={X:minibatch_X,Y:minibatch_Y})
#             _ , temp_loss, temp_accuracy = sess.run([optimizer,train_loss,accuracy],feed_dict={X:minibatch_X,Y:minibatch_Y})
            
#             if i % (BATCH_SIZE*15) == 0 and i >200:  # Record summaries and test-set accuracy
#                 summary = sess.run(merged_summary, feed_dict={X:minibatch_X,Y:minibatch_Y})
#                 writer.add_summary(summary, i + epoch*len(train_data))
            
#             #writer.add_summary(summary, i)
#             print("temp_loss: "+str(temp_loss))
            
#             if i % (BATCH_SIZE*50) == 0 and i>=1000:
#                 print("validating...")
#                 val_loss=0
#                 val_accuracy = 0
#                 val_batches = len(val_data)/50
#                 for v in range(0,len(val_data),50):
#                     val_x,val_y = prepare_data(val_data[v:v+BATCH_SIZE])
#                     temp_loss, temp_accuracy = sess.run([train_loss,accuracy],feed_dict={X:val_x,Y:val_y})
#                     val_accuracy+=temp_accuracy
#                     val_loss+=temp_loss
#                 print("validation loss:")
#                 print(val_loss/val_batches)
#                 print("validation accuracy: ")
#                 print(val_accuracy/val_batches)
#             print("temp_accuracy: "+str(temp_accuracy))
#     train_accuracy = accuracy.eval({X: train_data, Y: train_labels})
#     val_data, val_labels = prepare_data(val_data)
#     val_accuracy = accuracy.eval({X: val_data, Y: val_labels})
# #     print("Train Accuracy:", train_accuracy)
#     print("Validation Accuracy:", val_accuracy)

creating model
reading vgg16 json file
creating layer :conv1_1
creating layer :conv1_2
creating layer :conv2_1
creating layer :conv2_2
creating layer :conv3_1
creating layer :conv3_2
creating layer :conv3_3
creating layer :conv4_1
creating layer :conv4_2
creating layer :conv4_3
creating layer :conv5_1
creating layer :conv5_2
creating layer :conv5_3
creating layer : fc6
creating layer : fc7
creating layer : fc8
model completed.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See tf.nn.softmax_cross_entropy_with_logits_v2.

optimizer set
accuracy set
session created.
initializing global variables
